In [1]:
win_size = 9

In [2]:
import sys
sys.path.append('../')
from time import sleep
from models.attention_series import Grade_regressor
import spectral
from configs.training_cfg import device
import torch
import ast
import numpy as np
from PIL import Image
import pandas as pd
from spectral import imshow
from matplotlib import rcParams
import seaborn as sns
from matplotlib.pyplot import MultipleLocator
import matplotlib.pyplot as plt

spectral.settings.envi_support_nonlowercase_params = True

rcParams['figure.figsize'] = 11,19

pre_list = []
erro_list = []

In [ ]:
# 测试样本绝对误差
for k in range(1):
    model = Grade_regressor().to(device).eval()
    model.load_state_dict(torch.load("..\\ckpt\\替换回RI_relu_41500.pt".format(k+1)))
    result = []
    err = 0
    err_count = 0 
    pool = torch.nn.AvgPool2d(win_size,win_size)
    mask_rgb_values = [[255,242,0],[34,177,76],[255,0,88],[184,61,186]]

    spec_id = [9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64]    # 选择评估的成像光谱图片编号


    for id in spec_id:
        img = spectral.envi.open("E:\\d盘备份\\可见光部分\\spectral_data\{}-Radiance From Raw Data-Reflectance from Radiance Data and Measured Reference Spectrum.bip.hdr".format(id))

        # 根据模型使用波段选择
        # img_data = torch.Tensor(img.asarray()/6000)[:,:,:-4]
        img_data = torch.Tensor(img.asarray()/6000)[:,:,:]
        mask = np.array(Image.open("E:\\d盘备份\\可见光部分\\spectral_data\\{}-Radiance From Raw Data-Reflectance from Radiance Data and Measured Reference Spectrum.bip.hdr_mask.png".format(id)))
        if mask.shape[2] == 4:
            mask = mask[:,:,:-1]
        gt_TFe = ast.literal_eval(img.metadata['gt_TFe'])
        gt = torch.Tensor(gt_TFe)
        gts = gt.__len__()

        with torch.no_grad():
            img_data = pool(img_data.permute(2,0,1)).permute(1,2,0)
            row,col,_ = img_data.shape
            heat_map = []

            for i in range(row):
                heat_map.append(model(img_data[i].to(device)).squeeze(1).unsqueeze(0).to("cpu"))   # 只评估Tfe
                torch.cuda.empty_cache()

        heat_map = torch.cat(heat_map, dim=0)

        # 可视化
        plt.figure()#防止在上一张图上重新绘制
        hm = sns.heatmap(heat_map.numpy(), cmap="viridis", xticklabels=False, yticklabels=False)
        hm.figure.savefig("新可见光relu/{}号预测结果可视化.jpg".format(id))

        predict_sum = torch.zeros(gts)
        pixel_count = torch.ones(gts)/1e5

        for r in range(row):
            for c in range(col):
                for i in range(gts):
                    if mask[r*win_size+win_size//2,c*win_size+win_size//2].tolist() == mask_rgb_values[i]:
                        predict_sum[i] += heat_map[r,c] 
                        pixel_count[i] += 1

        prediction = predict_sum / pixel_count * 100

        err_list = ((prediction-gt)).tolist()
        pred_list = (prediction).tolist()
        for i in range(err_list.__len__()):
            erro_list.append(round(err_list[i],2))
            pre_list.append(round(pred_list[i], 2))
            

    # avg_err = err/(err_count)
    # print("{}k  avgmse:{}".format(k+1, avg_err))

In [4]:
erro_list

[-4.08,
 7.58,
 -5.34,
 -0.53,
 2.8,
 -7.5,
 -1.64,
 -2.34,
 4.54,
 -3.22,
 3.28,
 9.06,
 -1.62,
 -1.86,
 0.16,
 1.4,
 -1.28,
 -3.11,
 -2.95,
 -4.73,
 1.94,
 0.07,
 -3.58,
 3.42,
 -9.27,
 -5.82,
 1.19,
 2.4,
 -0.48,
 4.49,
 -3.57,
 -3.33,
 -3.46,
 -1.87,
 -3.89,
 4.33,
 -7.97,
 2.59,
 -9.16,
 0.95,
 -3.94,
 -4.65,
 5.19,
 -6.7,
 -6.45,
 -0.34,
 4.65,
 -0.91,
 -1.87,
 2.34,
 -1.69,
 -5.35,
 4.89,
 7.0,
 3.28,
 4.16,
 -7.2,
 -2.44,
 0.9,
 -0.04,
 -4.94,
 -4.32,
 -4.24,
 -0.2,
 0.91,
 -6.04,
 -0.41,
 3.87,
 -4.61,
 1.13,
 -3.45,
 -8.13,
 0.61,
 -6.49,
 7.38,
 2.79,
 0.57,
 5.01,
 -5.08,
 -0.77,
 -3.5,
 4.62,
 -6.75,
 1.21,
 7.06,
 -2.37,
 -9.7,
 -4.1,
 -2.44,
 -0.45,
 -4.61,
 4.72,
 -2.56,
 -0.16,
 -5.14,
 -3.5,
 -1.22,
 -5.91,
 -8.72,
 -7.36,
 -8.44,
 2.78,
 2.62,
 -2.41,
 6.94,
 -7.08,
 -3.03,
 6.22,
 -8.08,
 -2.62,
 6.36,
 -4.83,
 0.35,
 2.5,
 4.84,
 7.98,
 -2.04,
 -3.56,
 3.68,
 -2.2,
 -2.79,
 2.45,
 -2.93,
 0.82,
 5.62,
 -3.92,
 3.74,
 -6.45,
 0.6,
 -6.25,
 1.68,
 -2.95,
 -2.68,
 -3.

In [ ]:
pre_list

[30.26,
 32.86,
 31.42,
 34.92,
 30.87,
 30.07,
 26.9,
 19.26,
 23.82,
 23.36,
 19.19,
 23.45,
 32.93,
 23.44,
 32.2,
 29.26,
 23.81,
 30.44,
 29.38,
 24.2,
 30.55,
 29.87,
 26.04,
 31.56,
 28.34,
 27.73,
 23.45,
 27.9,
 25.18,
 23.22,
 13.66,
 30.7,
 16.24,
 13.94,
 31.43,
 12.39,
 10.65,
 19.73,
 31.79,
 13.43,
 23.19,
 31.08,
 17.02,
 25.27,
 28.84,
 17.71,
 25.52,
 29.44,
 21.51,
 22.71,
 25.47,
 23.45,
 23.85,
 27.28,
 26.45,
 23.24,
 42.21,
 26.97,
 25.61,
 41.47,
 64.19,
 17.86,
 25.36,
 63.94,
 20.03,
 22.23,
 25.65,
 25.43,
 27.37,
 25.41,
 26.94,
 28.17,
 17.84,
 30.5,
 17.32,
 28.97,
 21.74,
 26.44,
 21.71,
 46.69,
 27.06,
 30.75,
 26.4,
 25.14,
 24.31,
 37.58,
 25.15,
 22.81,
 24.79,
 29.74,
 25.85,
 23.91,
 29.77,
 23.5,
 25.64,
 24.9,
 28.29,
 19.52,
 23.58,
 26.87,
 33.47,
 25.04,
 24.31,
 41.36,
 20.13,
 23.32,
 34.79,
 63.69,
 68.11,
 27.11,
 34.74,
 39.65,
 34.32,
 31.28,
 33.62,
 26.99,
 27.84,
 43.75,
 30.38,
 22.9,
 41.18,
 21.3,
 22.48,
 29.75,
 24.41,
 28.51,
 23